In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat Mar  4 07:31:02 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    58W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 103.2 MB/s eta 0:00:00


In [4]:
import os 
import numpy as np
import pandas as pd
import json
import warnings
import logging
import gc
import random
import math
import re
import ast
from tqdm import tqdm
from typing import Optional
from datetime import datetime
import pickle
import sklearn
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import jaccard_score, f1_score, accuracy_score, recall_score, precision_score, confusion_matrix
from sklearn.model_selection import train_test_split

In [5]:
import pickle
import torch
import transformers
from transformers import BertTokenizer, BertModel

from transformers import AutoTokenizer, AutoModel, AutoConfig




import torch.nn.functional as F
import numpy as np

import math
import pandas as pd
import pip
import re

import os.path as osp



import random

random.seed(123)

torch.manual_seed(123)
np.random.seed(123)
torch.cuda.manual_seed_all(123)
# from sklearn.metrics import jaccard_score, f1_score, accuracy_score, recall_score, precision_score, confusion_matrix


In [6]:
with open('/content/drive/MyDrive/Colab Notebooks/pakdd_baseline/data_dict.p', 'rb') as f:
  data_dict = pickle.load(f)

In [7]:
train_data = data_dict['train_data']
train_label = data_dict['train_label']
valid_data = data_dict['valid_data']
valid_label = data_dict['valid_label']
test_data = data_dict['test_data']
test_label = data_dict['test_label']

In [8]:
train_data[0]

'[PAT] : Hello! Doctor. I have got Skin lesion. I am a little worried about it. [SEP] [DOC] : Per your symptom, you may have [MASK]. '

In [9]:

tokenizer = AutoTokenizer.from_pretrained('microsoft/BiomedNLP-KRISSBERT-PubMed-UMLS-EL')
language_model = AutoModel.from_pretrained('microsoft/BiomedNLP-KRISSBERT-PubMed-UMLS-EL')

In [10]:
language_model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30525, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [11]:
tokenizer

BertTokenizerFast(name_or_path='microsoft/BiomedNLP-KRISSBERT-PubMed-UMLS-EL', vocab_size=30522, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [12]:
class CustomModel(torch.nn.Module):
  def __init__(self, hidden_dim, output_dim):
    super(CustomModel, self).__init__()

    self.custom_model = language_model
    self.dense = torch.nn.Linear(hidden_dim, output_dim)

  def forward(self, input_ids, attention_mask):
    model_out = self.custom_model(input_ids, attention_mask)['last_hidden_state'][:, 0, :]
    dense_out = self.dense(model_out)
    return dense_out  


In [13]:
model = CustomModel(hidden_dim = 768, output_dim = 90)
# model = CustomModel(hidden_dim = 1024, output_dim = 90)

In [14]:

print(tokenizer.all_special_tokens)
print(tokenizer.all_special_ids)
p = {
        'additional_special_tokens' : ['[DOC]', '[PAT]']
    }

tokenizer.add_special_tokens(p)
print(tokenizer.convert_tokens_to_ids('[DOC]'))
print(tokenizer.convert_tokens_to_ids('[PAT]'))
print(tokenizer.all_special_tokens)
print(tokenizer.all_special_ids)
print(language_model.resize_token_embeddings(len(tokenizer)))

['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]']
[1, 3, 0, 2, 4]
30525
30526
['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]', '[DOC]', '[PAT]']
[1, 3, 0, 2, 4, 30525, 30526]
Embedding(30527, 768)


In [15]:
tokenizer

BertTokenizerFast(name_or_path='microsoft/BiomedNLP-KRISSBERT-PubMed-UMLS-EL', vocab_size=30522, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]', 'additional_special_tokens': ['[DOC]', '[PAT]']})

In [17]:
enc_train = tokenizer(train_data, padding=True, max_length=512, truncation=True, return_tensors = 'pt')
enc_valid = tokenizer(valid_data, padding=True, max_length=512, truncation=True, return_tensors = 'pt')
enc_test = tokenizer(test_data, padding=True, max_length=512, truncation=True, return_tensors = 'pt')

# enc_train = tokenizer(train_data,  max_length=512, truncation=True, return_tensors = 'pt')
# enc_valid = tokenizer(valid_data,  max_length=512, truncation=True, return_tensors = 'pt')
# enc_test = tokenizer(test_data,  max_length=512, truncation=True, return_tensors = 'pt')

# enc_train = tokenizer(train_data,  return_tensors = 'pt')
# enc_valid = tokenizer(valid_data,  return_tensors = 'pt')
# enc_test = tokenizer(test_data,  return_tensors = 'pt')



In [18]:
train_label = torch.tensor(train_label)
print(train_label.shape)

valid_label = torch.tensor(valid_label)
print(valid_label.shape)

test_label = torch.tensor(test_label)
print(test_label.shape)

torch.Size([957])
torch.Size([137])
torch.Size([273])


In [19]:
class MedicalDataset(Dataset):
  def __init__(self, input_ids, attention_mask, labels):
    self.input_ids = input_ids
    self.attention_mask = attention_mask
    self.labels = labels

  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, idx):
    return self.input_ids[idx], self.attention_mask[idx], self.labels[idx]    

In [20]:
train_loader = DataLoader(MedicalDataset(enc_train['input_ids'], enc_train['attention_mask'], train_label), batch_size = 16, shuffle = True )
valid_loader = DataLoader(MedicalDataset(enc_valid['input_ids'], enc_valid['attention_mask'], valid_label), batch_size = 16, shuffle = False)
test_loader = DataLoader(MedicalDataset(enc_test['input_ids'], enc_test['attention_mask'], test_label), batch_size = 16, shuffle = False)

In [21]:
model

CustomModel(
  (custom_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30527, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
       

In [22]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [23]:
model = model.to(device)

In [24]:
total_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_parameters/1e6

109.55529

In [25]:
cnt = 0
for name, parma in model.named_parameters():
  print(cnt, " : ", name)
  cnt+=1

total_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_parameters/1e6

0  :  custom_model.embeddings.word_embeddings.weight
1  :  custom_model.embeddings.position_embeddings.weight
2  :  custom_model.embeddings.token_type_embeddings.weight
3  :  custom_model.embeddings.LayerNorm.weight
4  :  custom_model.embeddings.LayerNorm.bias
5  :  custom_model.encoder.layer.0.attention.self.query.weight
6  :  custom_model.encoder.layer.0.attention.self.query.bias
7  :  custom_model.encoder.layer.0.attention.self.key.weight
8  :  custom_model.encoder.layer.0.attention.self.key.bias
9  :  custom_model.encoder.layer.0.attention.self.value.weight
10  :  custom_model.encoder.layer.0.attention.self.value.bias
11  :  custom_model.encoder.layer.0.attention.output.dense.weight
12  :  custom_model.encoder.layer.0.attention.output.dense.bias
13  :  custom_model.encoder.layer.0.attention.output.LayerNorm.weight
14  :  custom_model.encoder.layer.0.attention.output.LayerNorm.bias
15  :  custom_model.encoder.layer.0.intermediate.dense.weight
16  :  custom_model.encoder.layer.0.inte

In [27]:
cnt = 0
for name, param in model.named_parameters():
  if(cnt>=199):
  # if(cnt>=391):  
    param.requires_grad = True
    print(cnt, " : ", name)
  else:
    param.requires_grad = False 

  cnt+=1   


199  :  dense.weight
200  :  dense.bias


In [28]:
total_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_parameters/1e6

0.06921

In [29]:
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)
criterion = torch.nn.CrossEntropyLoss()


In [30]:
def train_epoch(model, data_loader):
  model.train()
  epoch_train_loss = 0.0

  for step, batch in enumerate(tqdm(data_loader)):
    optimizer.zero_grad()

    batch = tuple(t.to(device) for t in batch)

    input_ids, attention_mask, labels = batch

    output = model(input_ids = input_ids,
                   attention_mask = attention_mask)
    
    loss = criterion(output, labels.long())

    epoch_train_loss += loss.item()

    loss.backward()
    optimizer.step()

  print("Epoch train loss : ", epoch_train_loss)    


  

In [31]:
def valid_epoch(model, data_loader):
  model.eval()
  predictions = []
  gold = []

  valid_loss = 0.0

  with torch.no_grad():
    for step, batch in enumerate(tqdm(data_loader)):
      batch = tuple(t.to(device) for t in batch)

      input_ids, attention_mask, labels = batch

      output = model(input_ids = input_ids,
                   attention_mask = attention_mask)
    
      loss = criterion(output, labels.long())

      valid_loss += loss.item()

      pred = output.argmax(dim = -1)

      predictions.extend(pred.tolist())
      gold.extend(labels.tolist())

  return valid_loss, predictions, gold    



In [32]:
def test_epoch(model, data_loader):
  model.eval()
  predictions = []
  gold = []

  with torch.no_grad():
    for step, batch in enumerate(tqdm(data_loader)):
      batch = tuple(t.to(device) for t in batch)

      input_ids, attention_mask, labels = batch

      output = model(input_ids = input_ids,
                   attention_mask = attention_mask)
      
      pred = output.argmax(dim = -1)

      predictions.extend(pred.tolist())
      gold.extend(labels.tolist())


  return predictions, gold    





In [33]:
def train_and_validation(model, train_loader, valid_loader):
  best_acc = 0.0
  

  for epoch in range(50):

    print("\n=============Epoch : ", epoch)
    train_epoch(model, train_loader)
    valid_loss, valid_pred, valid_gold = valid_epoch(model, valid_loader)

    print("Length of predictions : ", len(valid_pred))
    print("Length of gold : ", len(valid_gold))
    print("Valid loss : ", valid_loss)
    
    print("\nAccuracy\n", accuracy_score(valid_gold, valid_pred))
    print("\nF1 score\n", f1_score(valid_gold, valid_pred, average = 'macro'))
    print("\nJaccard similarity\n", jaccard_score(valid_gold, valid_pred, average = 'macro')) 

    curr_acc = accuracy_score(valid_gold, valid_pred)

    print('Curr acc : ', curr_acc)
    print('Best acc : ', best_acc)

    if(curr_acc > best_acc):
      best_acc = curr_acc

      
      torch.save(model.state_dict(), '/content/drive/MyDrive/Colab Notebooks/pakdd_baseline/saved_model/krissbert/best_model_epoch_'+str(epoch)+'_best_acc_'+str(int(best_acc*100))+'_.pt')

      print("model saved\n")

In [34]:
train_and_validation(model, train_loader, valid_loader)


=============Epoch :  0


100%|██████████| 60/60 [00:08<00:00,  6.76it/s]


Epoch train loss :  272.18552350997925


100%|██████████| 9/9 [00:00<00:00, 15.64it/s]


Length of predictions :  137
Length of gold :  137
Valid loss :  38.63605451583862

Accuracy
 0.072992700729927

F1 score
 0.04765234765234765

Jaccard similarity
 0.03404761904761904
Curr acc :  0.072992700729927
Best acc :  0.0
model saved


=============Epoch :  1


...

Length of predictions :  137
Length of gold :  137
Valid loss :  13.631716847419739

Accuracy
 0.5547445255474452

F1 score
 0.41877248677248674

Jaccard similarity
 0.3507936507936508
Curr acc :  0.5547445255474452
Best acc :  0.583941605839416

=============Epoch :  49


100%|██████████| 60/60 [00:05<00:00, 10.35it/s]


Epoch train loss :  50.22205227613449


100%|██████████| 9/9 [00:00<00:00, 15.67it/s]

Length of predictions :  137
Length of gold :  137
Valid loss :  13.728794693946838

Accuracy
 0.5328467153284672

F1 score
 0.39909442409442414

Jaccard similarity
 0.33345543345543344
Curr acc :  0.5328467153284672
Best acc :  0.583941605839416


In [35]:
path = '/content/drive/MyDrive/Colab Notebooks/pakdd_baseline/saved_model/krissbert/best_model_epoch_42_best_acc_58_.pt'

In [36]:
model.load_state_dict(torch.load(path))

<All keys matched successfully>

In [37]:
test_pred, test_gold = test_epoch(model, test_loader)

print("\nAccuracy\n", accuracy_score(test_gold, test_pred))
print("\nF1 score\n", f1_score(test_gold, test_pred, average = 'macro'))
print("\nJaccard similarity\n", jaccard_score(test_gold, test_pred, average = 'macro')) 

100%|██████████| 18/18 [00:01<00:00,  9.47it/s]


Accuracy
 0.5714285714285714

F1 score
 0.49774281904963724

Jaccard similarity
 0.4091548602912239
